In [1]:
!nvidia-smi

Mon Apr  3 13:57:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0F:00.0 Off |                  N/A |
| 23%   28C    P8     9W / 250W |      0MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# # mount driver
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/CardiacSeg/eda/Network
#
# #install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# !pip install ray
# %matplotlib inline

In [2]:
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.ssl_head import SSLHead

# sync python module
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.4
args.depths = [4, 4, 8, 4]
net = None
del net
net = network('unetmnx', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

model: unetmnx
[3, 3, 9, 3]


The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


                                                            Kernel Shape  \
Layer                                                                      
0_encoder0.layer.conv1.Conv3d_conv                      [1, 24, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                                  -   
2_encoder0.layer.LeakyReLU_lrelu                                       -   
3_encoder0.layer.conv2.Conv3d_conv                     [24, 24, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                                  -   
5_encoder0.layer.conv3.Conv3d_conv                      [1, 24, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                                  -   
7_encoder0.layer.LeakyReLU_lrelu                                       -   
8_inc.Conv3d_0                                          [1, 96, 4, 4, 4]   
9_inc.Conv3d_0                                          [1, 96, 4, 4, 4]   
10_inc.BatchNorm3d_1                                                [96]   
11_inc.Batch

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder0.layer.conv1.Conv3d_conv,"[1, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",648.0,1.358954e+09
1_encoder0.layer.InstanceNorm3d_norm1,-,"[1, 24, 128, 128, 128]",NaN,NaN
2_encoder0.layer.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
3_encoder0.layer.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
4_encoder0.layer.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN
...,...,...,...,...
463_decoder1.conv_block.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN
464_decoder1.conv_block.conv3.Conv3d_conv,"[48, 24, 1, 1, 1]","[1, 24, 128, 128, 128]",1152.0,2.415919e+09
465_decoder1.conv_block.InstanceNorm3d_norm3,-,"[1, 24, 128, 128, 128]",NaN,NaN


In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

model: unetcnx_x2


/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


                                                           Kernel Shape  \
Layer                                                                     
0_encoder0.layer.conv1.Conv3d_conv                     [1, 24, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                                 -   
2_encoder0.layer.LeakyReLU_lrelu                                      -   
3_encoder0.layer.conv2.Conv3d_conv                    [24, 24, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                                 -   
5_encoder0.layer.conv3.Conv3d_conv                     [1, 24, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                                 -   
7_encoder0.layer.LeakyReLU_lrelu                                      -   
8_inc.Conv3d_0                                         [1, 48, 2, 2, 2]   
9_inc.Conv3d_0                                         [1, 48, 2, 2, 2]   
10_inc.LayerNorm3d_1                                               [48]   
11_inc.LayerNorm3d_1     

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder0.layer.conv1.Conv3d_conv,"[1, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",648.0,1.358954e+09
1_encoder0.layer.InstanceNorm3d_norm1,-,"[1, 24, 128, 128, 128]",NaN,NaN
2_encoder0.layer.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
3_encoder0.layer.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
4_encoder0.layer.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN
...,...,...,...,...
387_decoder1.decoder_block.0.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
388_decoder1.decoder_block.0.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
389_decoder1.decoder_block.0.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN


## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))